In [1]:
import requests
import threading,time
from bs4 import BeautifulSoup
import pandas as pd
import math

#from konlpy.tag import Okt

In [4]:
# 댓글 수집 자동화

url ="https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=188909&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}"

response = requests.get(url.format(1))
soup = BeautifulSoup(response.text,"html.parser")
total_score = soup.select_one("body > div > div > div.score_total > strong > em").text
total_score =total_score.replace(',','')
print('total_score:',total_score)
totalPage = math.ceil(int(total_score)/ 10) # 네이버 페이지를 10페이지를 기준으로 나눔

#############################################################

def find_comment(page = 1):
    response = requests.get(url.format(page))
    soup = BeautifulSoup(response.text,"html.parser")
    tag = soup.find("div",class_='score_result')
    tagli = tag.find_all("li")
    
    #평점과 댓글을 입력할 리스트를 생성한다.
    score,text = [],[]
    for e in tagli:
        if int(e.em.text) >= 8:
            score.append(1)
            text.append(e.p.get_text(" ",strip=True))
        elif int(e.em.text) <= 5:
            score.append(0)
            text.append(e.p.get_text(" ", strip=True))

    return score,text    

total_score: 17444


In [9]:
def searchNaver(totalPage):
    # 1페이지부터
    sv, tv = [], []
    for i in range(1, totalPage):
        time.sleep(0.1)
        print("Count :", i, end="\r")
        s, t = find_comment(i)

        # 파이썬에서는 배열끼리 덧셈 연산을 하면 두개가 하나로 합쳐진다.
        sv += s
        tv += t
    
    df = pd.DataFrame([sv, tv]).T
    
    # 컬럼 지정
    df.columns = ['score', 'text']
    print(df)
    
    # CSV파일로 저장하기
    df.to_csv('comment.csv')

In [10]:
t1 = threading.Thread(target=searchNaver,args=(totalPage,))
t1.start()
print("Test Code!")

Test Code!
      score                                               text
0         0  관람객 배우들 연기는 참 좋은데, 스토리 개연성이 전혀 없고, 미국 대통령을 누굴 ...
1         0                                      보지마세요.. 진심입니다
2         0                          관람객 기대한것보다는 너무 하네요 실망 대실망
3         1                               관람객 진짜 부함장님이 젤 멋짐 ^^
4         0                        관람객 그냥그랬어요.올해본영화중엔 제일별로였어요.
5         0                                        너무한다는생각밖ㅇ니ㅣ
6         0                                진짜 시간낭비함 나만 당할 수 없다
7         0  어떻게 이런 영화를 만들었는ㅈ진짜 재미없음 돈아까움영화 보는중에 졸다가 폰들어 글쓰...
8         0  이야..영화를 이렇게 못만드나..마지막부분에 신정근배우 활약하는 부분 외에는 진짜 ...
9         0  관람객 한미 두 대통령이 북한에 있더라도 그렇게 쉽게 납치됨장르가 드라마 액션인데 ...
10        0                                    시대착오적인 반일 영화 ㅠㅠ
11        0                              개연성이 1도없고 장르가 뭔지 모르겠음
12        0                                  양심있는 박평식씨는 평을 안했음
13        0  ㅋㅋ그래도 명색이 정치가 가미된 액션인데 정치와 세계정세를 1도 모르는 애가 너무 ...
14        0                                 